In [35]:
!pip install python-dotenv

In [36]:
import dotenv
import os
import json
import requests
from pprint import pprint

dotenv.load_dotenv()

True

In [41]:
# ['*'] for all the details or from ['name', 'id', 'types', 'nationalPhoneNumber', 'internationalPhoneNumber', 'formattedAddress', 'addressComponents', 'plusCode', 'location', 'viewport', 'rating', 'googleMapsUri', 'websiteUri', 'regularOpeningHours', 'utcOffsetMinutes', 'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount', 'iconMaskBaseUri', 'iconBackgroundColor', 'displayName', 'primaryTypeDisplayName', 'takeout', 'delivery', 'dineIn', 'curbsidePickup', 'servesBreakfast', 'servesLunch', 'servesDinner', 'servesBeer', 'servesWine', 'servesVegetarianFood', 'currentOpeningHours', 'primaryType', 'shortFormattedAddress', 'editorialSummary', 'reviews', 'photos', 'outdoorSeating', 'menuForChildren', 'servesDessert', 'servesCoffee', 'goodForChildren', 'restroom', 'goodForGroups', 'paymentOptions', 'parkingOptions', 'accessibilityOptions']
PLACE_DETAILS_FIELDS = [
    '*',
    # 'name', 'id', 'types',
    # 'formattedAddress','addressComponents',
    # 'location', 'rating', 
]
DATA_PATH = 'data/'

if not os.path.exists(os.path.join(DATA_PATH,'places')):
    os.makedirs(os.path.join(DATA_PATH,'places'))

## Fetch places

In [13]:
def fetch_places(data, fields: list = ['places.id']):
    headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": os.getenv("GMAPS_API_KEY"),
            "X-Goog-FieldMask": ",".join(fields)
    }
    res = requests.post("https://places.googleapis.com/v1/places:searchNearby", json=data, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        raise Exception(f"Error while fetching places with status {res.status_code}: {res.text}")

In [27]:
def fetch_place_details(place_id, fields: list = PLACE_DETAILS_FIELDS):
    headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": os.getenv("GMAPS_API_KEY"),
            "X-Goog-FieldMask": ",".join(fields)
    }
    res = requests.get("https://places.googleapis.com/v1/places/"+place_id, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        raise Exception(f"Error while fetching place details with status {res.status_code}: {res.text}")

In [42]:
def save_place(place, prefix=''):
    path = os.path.join(DATA_PATH, 'places', prefix+place['id']+'.json')
    with open(path, 'w') as f:
        json.dump(place, f)
    return path

In [75]:
def get_places(*place_params, detail_fields: list = PLACE_DETAILS_FIELDS, save_prefix='', ignore_if_exists=True):
    places_ids = []
    for pparams in place_params:
        places = fetch_places(pparams)
        places_ids.extend(map(lambda p:p['id'], places['places']))
    
    places_ids = list(set(places_ids))
    
    for place_id in places_ids:
        if ignore_if_exists and os.path.exists(os.path.join(DATA_PATH, 'places', save_prefix+place_id+'.json')):
            yield (place_id, None, None)
            continue

        try:
            place = fetch_place_details(place_id, detail_fields)
        except Exception as e:
            yield (place_id, e, None)
            continue

        try:
            save_path = save_place(place, save_prefix)
            yield (place_id, place, save_path)
        except Exception as e:
            yield (place_id, place, e)


In [81]:
padova_points = [{
        "latitude": 45.416668,
        "longitude": 11.866667
        },
        {
        "latitude": 45.4285770245207,
        "longitude": 11.887496970211265
        },
        {
        "latitude": 45.4061953606289, 
        "longitude": 11.877509545938512,
        },
        {
        "latitude": 45.40928596604953, 
        "longitude": 11.910659904660807,
        },
        {
        "latitude": 45.40498454875341, 
        "longitude": 11.856252843378272,
        },
        {
        "latitude": 45.39238427909031, 
        "longitude": 11.859049123320446,
        },
        {
        "latitude": 45.380370067550885,
        "longitude": 11.883652085236182,
        },
        {
        "latitude": 45.40928596604953, 
        "longitude": 11.910659904660807,
        },
        {
        "latitude": 45.3842376759648, 
        "longitude": 11.84967310315375,
        },
    
]

restaurant_params = [
    {
        "includedTypes": ["restaurant"],
        "locationRestriction": {
            "circle": {
                "center": p,
                "radius": 3000
            },
        }
    }
  for p in padova_points
]


restaurants = get_places(*restaurant_params, save_prefix='restaurant-', ignore_if_exists=True)

total_ids = 0
new_ids = 0
for place_id, place, save_path in restaurants:
    total_ids += 1

    if place is None:
        # it already exists
        continue
    if isinstance(place, Exception):
        print("Error while fetching place details with id:", place_id, "error:", place)
        continue
        
    if isinstance(save_path, str):
        print("New place found and saved with id:", place_id, "at", save_path)
        new_ids += 1
    elif isinstance(save_path, Exception):
        print("New place found with id:", place_id, "but COULD NOT SAVE IT, error:", save_path)
    else:
        print("New place found with id:", place_id, "but COULD NOT SAVE IT")
    

print("New places found:", new_ids, "/", total_ids)
        


New place found and saved with id: ChIJScQPg6_bfkcRCjD2IcCbQVI at data/places/restaurant-ChIJScQPg6_bfkcRCjD2IcCbQVI.json
New place found and saved with id: ChIJV2M7GFPafkcRM93CmDUQpq4 at data/places/restaurant-ChIJV2M7GFPafkcRM93CmDUQpq4.json
New place found and saved with id: ChIJpTWHmbzbfkcRdRu-nZfYwIY at data/places/restaurant-ChIJpTWHmbzbfkcRdRu-nZfYwIY.json
New place found and saved with id: ChIJ_0KcCADbfkcRZ8PDOxP_nGk at data/places/restaurant-ChIJ_0KcCADbfkcRZ8PDOxP_nGk.json
New place found and saved with id: ChIJAYYtjDLbfkcRbhEnwoYqiS0 at data/places/restaurant-ChIJAYYtjDLbfkcRbhEnwoYqiS0.json
New place found and saved with id: ChIJ-yQnyhPbfkcRAHXgsEK60pc at data/places/restaurant-ChIJ-yQnyhPbfkcRAHXgsEK60pc.json
New place found and saved with id: ChIJIWm-lxXbfkcR2SZ0UBpubvk at data/places/restaurant-ChIJIWm-lxXbfkcR2SZ0UBpubvk.json
New place found and saved with id: ChIJ_aCgVVbafkcR41roqUQj1BM at data/places/restaurant-ChIJ_aCgVVbafkcR41roqUQj1BM.json
New place found and save

## Read Places and Reviews

In [62]:
def read_places(prefix='', limit=None):
    dirpath = os.path.join(DATA_PATH, 'places')
    i = 0
    for p in os.scandir(dirpath):
        if not p.is_file() or not p.name.startswith(prefix):
            yield (p.name, None)
            continue
        
        i += 1
        if limit is not None and i > limit:
            break

        with open(p.path) as f:
            try:
                yield (p.name, json.load(f))
            except Exception as e:
                yield (p.name, e)

In [70]:
def read_reviews(places_iter, limit=None):
    i = 0
    for path, place in places_iter:
        for review in place['reviews']:
            i += 1
            if limit is not None and i > limit:
                break
            yield (place['id'], review)

In [82]:
i = 0
for r in read_reviews(read_places()):
    print(r)
    i += 1
print(i)

('ChIJpXSgrsDbfkcRzf_5kCMmrZI', {'name': 'places/ChIJpXSgrsDbfkcRzf_5kCMmrZI/reviews/ChdDSUhNMG9nS0VJQ0FnSUNUdnYyMnBBRRAB', 'relativePublishTimeDescription': 'in the last week', 'rating': 4, 'text': {'text': 'Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner', 'languageCode': 'en'}, 'originalText': {'text': 'Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner', 'languageCode': 'en'}, 'authorAttribution': {'displayName': 'Henk Vyncke', 'uri': 'https://www.google.com/maps/contrib/103733968569168024105/reviews', 'photoUri': 'https://lh3.googleusercontent.com/a-/ALV-UjWCd4R3VchPubvMkIA1_mlgDEPMEWrIbv9mprhaMZAbDjChvTtS=s128-c0x00000000-cc-rp-mo-ba4'}, 'publishTime': '2024-05-15T09:37:18Z'})
('ChIJpXSgrsDbfkcRzf_5kCMmrZI', {'name': 'places/ChIJpXSgrsDbfkcRzf_5kCMmrZI/reviews/ChZDSUhNMG9nS0VJQ0FnSU